In [19]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from datetime import datetime as dt
import logging
import json
import psycopg2
from psycopg2 import sql
import uuid

# Generate unique BATCH_ID for this ETL run
batch_id = str(uuid.uuid4())

world_bank_url = 'https://en.wikipedia.org/wiki/List_of_largest_banks'
exchange_rates_url = 'https://www.irs.gov/individuals/international-taxpayers/yearly-average-currency-exchange-rates'

headers_logs_df = ["Log Phase", "Message", "Datetime", "Batch ID"]
headers_world_bank = ["Bank name", "Market cap (US$ billion)", "Last Modified Date"]
headers_exchange_rates = ["Country", "Currency", "Exchange Rate", "Year"]


In [20]:
logs_df = pd.DataFrame(columns=headers_logs_df)

logging.basicConfig(level=logging.INFO, format='%(message)s')

def log_progress(phase, message):
    global logs_df
    current_time = dt.now().strftime('%Y-%m-%d %H:%M:%S')
    new_log = pd.DataFrame([[phase, message, current_time, batch_id]], columns=headers_logs_df)
    logs_df = pd.concat([logs_df, new_log], ignore_index=True)
    logging.info(f"{current_time} - {batch_id} - {phase} - {message}")

log_progress("Initialization", "Logging Initialized")

2024-07-23 07:09:53 - bdff4dc8-95e8-4984-823e-f5c706ad33f6 - Initialization - Logging Initialized


In [21]:
def extract_world_bank_data(url):
    extract_world_data_phase = "Extraction: World Bank Data"
    try:
        log_progress(extract_world_data_phase, "Starting extraction of world bank data")
        log_progress(extract_world_data_phase, f"Connecting to URL: {url}")
        response = requests.get(url)
        if response.status_code != 200:
            log_progress(extract_world_data_phase, f"Failed to connect to {url} status code: {response.status_code}")
            raise Exception(f"Failed to retrieve data from {url}")
        log_progress(extract_world_data_phase, "Successfully connected to URL")
        
        soup = bs(response.content, 'html.parser')
        
        log_progress(extract_world_data_phase, "Extracting table content")        
        world_bank_data = soup.find_all('tbody')
        if len(world_bank_data) <= 2:
            raise Exception("Unexpected table structure")
        world_bank_data = world_bank_data[2]
        rows = world_bank_data.find_all('tr')
        
        log_progress(extract_world_data_phase, "Extracting last modified date")        
        exchange_rates_last_modif = soup.find(id = "footer-info-lastmod")
        lastmod_text = exchange_rates_last_modif.get_text()
        date_string = lastmod_text.split("on")[1].split(",")[0].strip()
        formatted_last_modified_date = dt.strptime(date_string, '%d %B %Y').strftime('%Y-%m-%d')
        
        data = [
            [
                row.find_all('td')[1].get_text().strip(),
                row.find_all('td')[2].get_text().strip(),
                formatted_last_modified_date
            ]
            for row in rows if len(row.find_all('td')) > 0
        ]
        
        df = pd.DataFrame(data, columns = headers_world_bank)
        df["Batch ID"] = batch_id
        log_progress(extract_world_data_phase, "Completed extraction of world bank data")
        
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 1000)
        pd.set_option('display.colheader_justify', 'center')

        return df
    except Exception as e:
        log_progress(extract_world_data_phase, f"Failed during extraction of world bank data: {e}")
        raise

def extract_exchange_rates_data(url):
    extract_exchange_rates_phase = "Extraction: Exchange Rates Data"
    try:
        log_progress(extract_exchange_rates_phase, "Starting extraction of exchange rates data")
        log_progress(extract_exchange_rates_phase, f"Connecting to URL: {url}")
        response = requests.get(url)
        if response.status_code != 200:
            log_progress(extract_exchange_rates_phase, f"Failed to connect to {url} status code: {response.status_code}")
            raise Exception(f"Failed to retrieve data from {url}")
        log_progress(extract_exchange_rates_phase, "Successfully connected to URL")
        
        soup = bs(response.content, 'html.parser')
        
        log_progress(extract_exchange_rates_phase, "Extracting year from header")
        exchange_rates_header = soup.find('thead')
        exchange_rates_header_data = exchange_rates_header.find('tr')
        headers = [exchange_rates_header.get_text().strip() for exchange_rates_header in exchange_rates_header_data.find_all('th')]
        year_column = headers[2]
        
        year_column = None
        for exchange_rates_header in headers:
            if exchange_rates_header.isdigit() and len(exchange_rates_header) == 4:
                year_column = exchange_rates_header
                break
        
        if year_column is None:
            log_progress(extract_exchange_rates_phase, f"Failed trying to extract year column from the header column")
            raise Exception("No year column found in the table headers")
        
        log_progress(extract_exchange_rates_phase, "Creating date format from the extracted year")        
        date_str = f"31-12-{year_column}"
        
        log_progress(extract_exchange_rates_phase, "Extracting content from the table")
        exchange_rates_data = soup.find('tbody')
        if not exchange_rates_data:
            log_progress(extract_exchange_rates_phase, "No table found on the page")
            raise Exception("No table found on the page")
        
        rows = exchange_rates_data.find_all('tr')
        if not rows:
            log_progress(extract_exchange_rates_phase, "No rows found in the table")
        
        data = [
            [
                row.find_all('td')[0].get_text().strip(),
                row.find_all('td')[1].get_text().strip(),
                row.find_all('td')[2].get_text().strip(),
                date_str
            ]
            for row in rows if len(row.find_all('td')) > 0
        ]
        
        df = pd.DataFrame(data, columns = headers_exchange_rates)
        df["Batch ID"] = batch_id
        log_progress(extract_exchange_rates_phase, "Completed extraction of exchange rates data")
        return df
    
    except Exception as e:
        log_progress(extract_exchange_rates_phase, f"Failed during extraction of {e}")

In [22]:
def load_db_config(config_path = 'Config DB/config.json'):
    with open(config_path, 'r') as config_file:
        return json.load(config_file)

In [55]:
def insert_log(cur, log_phase, message, log_datetime, batch_id):
    cur.execute("""
        CALL etl.insert_log(%s, %s, %s, %s)
    """, ([log_phase, message, log_datetime, batch_id]))

def load_to_db(df, db_connection_info, table_name, batch_id):
    try:
        log_progress("Loading", f"Starting loading data to {table_name} table in database")
        conn = psycopg2.connect(**db_connection_info)
        cur = conn.cursor()
        
        if table_name == 'world_bank_data':
            for _, row in df.iterrows():
                try:
                    cur.execute("""
                        CALL etl.insert_or_update_world_bank_data(%s, %s, %s, %s)    
                    """, (row['Bank name'], row['Market cap (US$ billion)'], row['Last Modified Date'], batch_id))
                    
                    # Log the action only if rows were affected
                    if cur.rowcount > 0:
                        insert_log(cur, 'Insert/Update', f"Processed bank {row['Bank name']}", dt.now().strftime('%Y-%m-%d %H:%M:%S'), batch_id)
                except Exception as e:
                    log_progress("Loading", f"Failed to insert/update to world bank data: {e}")
                    raise
            try:
                # Deactivate old records that are not in the current batch
                cur.execute("CALL etl.deactivate_bank_records(%s)", (batch_id,))
                # Log the deactivation
                insert_log(cur, 'Deactivate', 'Deactivated old world bank records', dt.now().strftime('%Y-%m-%d %H:%M:%S'), batch_id)
            except Exception as e:
                log_progress("Loading", f"Failed to insert/update to world bank data: {e}")
                raise
        
        elif table_name == 'exchange_rates':
            for _, row in df.iterrows():
                try:
                    cur.execute("""
                    CALL etl.insert_or_update_exchange_rates(%s, %s, %s, %s, %s)
                    """, (row['Country'], row['Currency'], row['Exchange Rate'], row['Year'], batch_id))
                
                    # Log the action only if rows were affected
                    if cur.rowcount > 0:
                        insert_log(cur, 'Insert/Update', f"Processed exchange rate for {row['Country']} - {row['Currency']}", 
                                dt.now().strftime('%Y-%m-%d %H:%M:%S'), batch_id)
                except Exception as e:
                    log_progress("Loading", f"Failed to insert/update to exchange rates data: {e}")
                    raise
        
        conn.commit()
        log_progress("Loading", f"Completed loading data to {table_name} table in database")
    
    except Exception as e:
        log_progress("Loading", f"Failed during loading data to {table_name} table in database {e}")
        raise
    
    finally:
        cur.close()
        conn.close()

In [58]:
def main():
    # Load database configuration
    db_connection_info = load_db_config()
    
    # Extract and Transform Data
    world_bank_df = extract_world_bank_data(world_bank_url)
    print('World Data Bank:\n', world_bank_df.head(20), '\n')

    exchange_rates_df = extract_exchange_rates_data(exchange_rates_url)
    print('Exchange Rates:\n', exchange_rates_df.head(10))
    
    # Load data into the database
    load_to_db(world_bank_df, db_connection_info, 'world_bank_data', batch_id)
    load_to_db(exchange_rates_df, db_connection_info, 'exchange_rates', batch_id)
    
    # Save Logs
    conn = psycopg2.connect(**db_connection_info)
    cur = conn.cursor()
    for _, row in logs_df.iterrows():
        try:
            insert_log(cur, row['Log Phase'], row['Message'], row['Datetime'], row['Batch ID'])
            conn.commit()
        except Exception as e:
            log_progress("Log Saving", f"Failed during saving logs to database: {e}")
            raise
    cur.close()
    conn.close()

if __name__ == "__main__":
    main()

2024-07-24 07:46:18 - bdff4dc8-95e8-4984-823e-f5c706ad33f6 - Extraction: World Bank Data - Starting extraction of world bank data
2024-07-24 07:46:18 - bdff4dc8-95e8-4984-823e-f5c706ad33f6 - Extraction: World Bank Data - Connecting to URL: https://en.wikipedia.org/wiki/List_of_largest_banks
2024-07-24 07:46:18 - bdff4dc8-95e8-4984-823e-f5c706ad33f6 - Extraction: World Bank Data - Successfully connected to URL
2024-07-24 07:46:18 - bdff4dc8-95e8-4984-823e-f5c706ad33f6 - Extraction: World Bank Data - Extracting table content
2024-07-24 07:46:18 - bdff4dc8-95e8-4984-823e-f5c706ad33f6 - Extraction: World Bank Data - Extracting last modified date
2024-07-24 07:46:18 - bdff4dc8-95e8-4984-823e-f5c706ad33f6 - Extraction: World Bank Data - Completed extraction of world bank data
2024-07-24 07:46:18 - bdff4dc8-95e8-4984-823e-f5c706ad33f6 - Extraction: Exchange Rates Data - Starting extraction of exchange rates data
2024-07-24 07:46:18 - bdff4dc8-95e8-4984-823e-f5c706ad33f6 - Extraction: Exchange

World Data Bank:
                  Bank name                 Market cap (US$ billion) Last Modified Date                Batch ID              
0                           JPMorgan Chase          551.03              2024-07-20      bdff4dc8-95e8-4984-823e-f5c706ad33f6
1                          Bank of America          288.96              2024-07-20      bdff4dc8-95e8-4984-823e-f5c706ad33f6
2  Industrial and Commercial Bank of China          249.28              2024-07-20      bdff4dc8-95e8-4984-823e-f5c706ad33f6
3                              Wells Fargo          208.41              2024-07-20      bdff4dc8-95e8-4984-823e-f5c706ad33f6
4               Agricultural Bank of China          207.79              2024-07-20      bdff4dc8-95e8-4984-823e-f5c706ad33f6
5                            Bank of China          171.35              2024-07-20      bdff4dc8-95e8-4984-823e-f5c706ad33f6
6                  China Construction Bank          166.19              2024-07-20      bdff4dc8-95e8-4984-

2024-07-24 07:46:19 - bdff4dc8-95e8-4984-823e-f5c706ad33f6 - Extraction: Exchange Rates Data - Successfully connected to URL
2024-07-24 07:46:19 - bdff4dc8-95e8-4984-823e-f5c706ad33f6 - Extraction: Exchange Rates Data - Extracting year from header
2024-07-24 07:46:19 - bdff4dc8-95e8-4984-823e-f5c706ad33f6 - Extraction: Exchange Rates Data - Creating date format from the extracted year
2024-07-24 07:46:19 - bdff4dc8-95e8-4984-823e-f5c706ad33f6 - Extraction: Exchange Rates Data - Extracting content from the table
2024-07-24 07:46:19 - bdff4dc8-95e8-4984-823e-f5c706ad33f6 - Extraction: Exchange Rates Data - Completed extraction of exchange rates data
2024-07-24 07:46:19 - bdff4dc8-95e8-4984-823e-f5c706ad33f6 - Loading - Starting loading data to world_bank_data table in database
2024-07-24 07:46:19 - bdff4dc8-95e8-4984-823e-f5c706ad33f6 - Loading - Completed loading data to world_bank_data table in database
2024-07-24 07:46:19 - bdff4dc8-95e8-4984-823e-f5c706ad33f6 - Loading - Starting loa

Exchange Rates:
       Country     Currency Exchange Rate     Year                   Batch ID              
0     Afghanistan  Afghani      82.635    31-12-2023  bdff4dc8-95e8-4984-823e-f5c706ad33f6
1         Algeria    Dinar     135.933    31-12-2023  bdff4dc8-95e8-4984-823e-f5c706ad33f6
2       Argentina     Peso     296.154    31-12-2023  bdff4dc8-95e8-4984-823e-f5c706ad33f6
3       Australia   Dollar       1.506    31-12-2023  bdff4dc8-95e8-4984-823e-f5c706ad33f6
4         Bahrain    Dinar       0.377    31-12-2023  bdff4dc8-95e8-4984-823e-f5c706ad33f6
5          Brazil     Real       4.994    31-12-2023  bdff4dc8-95e8-4984-823e-f5c706ad33f6
6          Canada   Dollar       1.350    31-12-2023  bdff4dc8-95e8-4984-823e-f5c706ad33f6
7  Cayman Islands   Dollar       0.833    31-12-2023  bdff4dc8-95e8-4984-823e-f5c706ad33f6
8           China     Yuan       7.075    31-12-2023  bdff4dc8-95e8-4984-823e-f5c706ad33f6
9         Denmark    Krone       6.890    31-12-2023  bdff4dc8-95e8-4984-